In [67]:
import os
main_path = "/Users/rinat/develop/datasets/cvat"
result_dir = "/Users/rinat/develop/datasets/coco_pole_ligth"

In [68]:
# находим zip файлы в cvat (экспорт без изображений только yolo формат) и распаковываем их
import zipfile
zips = [zip_ for zip_ in os.listdir(main_path) if zip_.find(".zip") > 0]
for zip_ in zips:
    zip_path = os.path.join(main_path, zip_)
    dirname = zip_.replace(".zip", "")
    with zipfile.ZipFile(zip_path,"r") as zip_ref:
        zip_ref.extractall(os.path.join(main_path,dirname))

In [69]:
import cv2

def save_all_frames(video_path, dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path,)
    if not cap.isOpened():
        return
    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)
    n = 0
    while True:
        ret, frame = cap.read()
        if ret:
            imname = '{}_{}.{}'.format(base_path, f"{n:06d}", ext)
            #print(imname)
            cv2.imwrite(imname, frame)
            n += 1
        else:
            print(video_path)
            return

In [70]:
# находим zip файлы и формируем директории с картинками в каждой ранее созданной папке

zips = [zip_ for zip_ in os.listdir(main_path) if zip_.find(".zip") > 0]
for zip_ in zips:
    dirname = zip_.replace(".zip", "")
    full_dirname = os.path.join(main_path, f"{dirname}/images/")
    video_path = os.path.join(main_path, f"{dirname}.left.avi") 
    save_all_frames(video_path, full_dirname, "frame", ext='jpg')

/Users/rinat/develop/datasets/cvat/ge4.740.007.left.avi


In [72]:
# список всех не нулевых файлов внутри каждой директории
all_txt_files = list()
for dir_ in os.listdir(main_path):
    if os.path.isdir(os.path.join(main_path, dir_)) is True:
        obj_train_data = os.path.join(os.path.join(main_path, dir_), "obj_train_data")
        files = [(os.path.join(obj_train_data, txt_file), txt_file) for txt_file in os.listdir(obj_train_data) if os.path.getsize(os.path.join(obj_train_data, txt_file))>0]
        all_txt_files += files

# находим все наше добро и переносим на уровень выше
# получаем вложенность
# coco_pole_ligth/images/train
# coco_pole_ligth/labels/train
train_im = os.path.join(os.path.join(result_dir,"images"),"train")
labels_im = os.path.join(os.path.join(result_dir,"labels"),"train")
os.makedirs(train_im, exist_ok=True)
os.makedirs(labels_im, exist_ok=True)

# копируем все в указанные выше директории
for file_path, file_name in all_txt_files:
    # разметка
    destination_txt = os.path.join(labels_im, file_name)
    os.popen(f'cp {file_path} {destination_txt}')
    # изображения
    destination_image = os.path.join(train_im, file_name.replace("txt","jpg"))
    source_im = file_path.replace("obj_train_data","images").replace("txt","jpg")
    print(source_im)
    os.popen(f'cp {source_im} {destination_image}')


/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000100.jpg
/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000098.jpg
/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000099.jpg
/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000094.jpg
/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000095.jpg
/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000097.jpg
/Users/rinat/develop/datasets/cvat/ge4.740.007/images/frame_000096.jpg


In [97]:
def readTxt(filepath, row, cols):
    res = dict()
    f = open(filepath, "r")
    Lines = f.readlines()
    key = 0
    for line in Lines:
        split_line = line.strip().split(" ")
        print(split_line)
        center_u = float(split_line[1]) * cols
        center_v = float(split_line[2]) * row
        wight_u = float(split_line[3]) * cols
        hight_v = float(split_line[4]) * row
        # формируем края 
        start_point = [center_u - wight_u/2, center_v - hight_v/2]
        end_point = [center_u + wight_u/2, center_v + hight_v/2]
        res[f"box_{key}"]  = {"class": int(split_line[0]), "start_point": start_point, "end_point" : end_point}
        key += 1
    f.close()
    return res


def getColor(class_):
    if class_ == 1:
        return (255, 0, 0)
    else:
        return (0, 255, 0)
     

    

In [100]:
# проверяем правильность разметки, визуализируя результат
# Window name in which image is displayed 
window_name = 'Image'
# Line thickness of 2 px 
thickness = 2

for name in os.listdir(train_im):
    if name.find("jpg") > 0:
        impath = os.path.join(train_im, name)
        image = cv2.imread(impath)
        r, c, _ = image.shape
        # читаем разметку
        label = impath.replace("images", "labels").replace("jpg","txt")
        results = readTxt(label, r, c)
        for key in results.keys():
            imclass = results[key]["class"]
            print(imclass)
            start_point = results[key]["imclass"]
            end_point = results[key]["imclass"]
            image = cv2.rectangle(image, start_point, end_point, color, thickness) 
        # Displaying the image  
        cv2.imshow(window_name, image)
        cv2.waitKey(1)
    cv2.destroyAllWindows()

['0', '0.545130', '0.250491', '0.041615', '0.046574']


KeyError: 'imclass'

In [ ]:
dsdsds